In [ ]:
# FIRST SCRIPT TO AUGMENT EACH IMAGE BY ROTATING 3 DIFFERENT WAYS; TO BE RUN IN PARALLEL IN BASH

from PIL import Image
import argparse
import os

# Define the argument parser; takes two arguments - path to jpeg, and output prefix which will be appended with rotation and .jpeg
parser = argparse.ArgumentParser(description='Transform an image and save the transformed images.')
parser.add_argument('input_path', type=str, help='Path to the input JPEG file.')
parser.add_argument('output_prefix', type=str, help='Prefix to use for output filenames.')

# Parse the arguments
args = parser.parse_args()

# Load the input image
with Image.open(args.input_path) as im:
    
    # Crop the right-hand third of the image vertically, because it is all black in all images
    width, height = im.size
    cropped_im = im.crop((0, 0, width - width // 3, height))

    # Resize the cropped image to 800x800
    resized_im = cropped_im.resize((800, 800))

    # Save the original transformed image
    resized_im.save(args.output_prefix + '_orig.jpg')

    # Create 3 additional rotated images and save them
    for angle in [90, 180, 270]:
        rotated_im = resized_im.rotate(angle)
        output_path = args.output_prefix + f'_{angle}.jpg'
        rotated_im.save(output_path)

In [ ]:
# SECOND SCRIPT, TO PROCESS THE DATA, SET UP THE TENSORS, AND WRITE THE TENSORS TO DISK AS NUMPY BINARIES

import os
import numpy as np
import pandas as pd
from PIL import Image
from collections import defaultdict
from sys import stderr

print('Reading patient data...',file=stderr)

patient_data = pd.read_excel('CMMD_clinicaldata_revision.xlsx',index_col='ID1')
patient_data.index += patient_data.LeftRight

all_data = defaultdict(dict)

# Iterate over the data frame and collect values associated with patient ID keys
for pat_id in patient_data.index:
    if 'D2' in pat_id: # Skip D2 patient IDs as they are a different dataset
        continue
    all_data[pat_id]['calc'] = 1 if patient_data.abnormality[pat_id] != 'mass' else 0
    all_data[pat_id]['mass'] = 1 if patient_data.abnormality[pat_id] != 'calcification' else 0
    all_data[pat_id]['classification'] = 1 if patient_data.classification[pat_id] == 'Malignant' else 0
    
# Free up memory
del patient_data

print('Reading image data...',file=stderr)

# Change to directory with augmented images
os.chdir('Augmented')

count = 0

# Iterate over all jpegs in directory and load them
for fname in os.listdir(os.getcwd()):
    if fname.split('.')[-1].lower() in ('.jpg','.jpeg'):
        continue
    count += 1
    print(f'\r{count}',end='')
    pat_id = ''.join(fname.split('.')[:2])
    img = Image.open(fname) # Load image
    img = np.array(img) # Convert to 2D array
    img = img.astype(np.float16) # Convert to 16bit float
    img /= 255 # Min/max normalizes pixel brightness values, which range from 0-255
    view = 'img_side' if '.S' in fname else 'img_top'
    rotation = fname.split('_')[-1].split('.')[0] # Value will be 90, 180, 270, or orig
    view += f'_{rotation}' # Values will be img_{side/top}_{90/180/270/orig}
    all_data[pat_id][view] = img # Add 2D array of min/max normalized pixel values to patient data dict

# Convert all_data to a dataframe
all_data_df = pd.DataFrame(all_data).T # Transpose so rows are patients

# Free up memory
del all_data

print('\nSetting up tensors...',file=stderr)

# For side images and top images, concatenate all rotations, in the same order
side_img_concat = pd.concat([all_data_df.img_side_orig, all_data_df.img_side_90, all_data_df.img_side_180, all_data_df.img_side_270])
top_img_concat = pd.concat([all_data_df.img_top_orig, all_data_df.img_top_90, all_data_df.img_top_180, all_data_df.img_top_270])

# Create stacked numpy arrays out of the concatenated images
side_img_tensor = np.stack(side_img_concat)
top_img_tensor = np.stack(top_img_concat)

# Repeat the labels 4x to account for the four rotations; label order will match
labels = np.repeat(all_data_df[['calc','mass','classification']].to_numpy().astype(np.float16), 4, axis=0)

# Print shape and data type of all tensors just to check for correctness
print('\nShapes:',file=stderr)
for x in [side_img_tensor,top_img_tensor]:
    print(x.shape,file=stderr)
print(labels.shape,file=stderr)
    
print('\nDTypes:',file=stderr)
for x in [side_img_tensor,top_img_tensor]:
    print(x.dtype,file=stderr)
print(labels.dtype,file=stderr)

# Get permuted indices to shuffle the tensors maintaining associations
num_samples = side_img_tensor.shape[0] # First value in shape is number of samples
permuted_indices = np.random.permutation(num_samples) # Shuffled array of ints from zero to num_samples

# use the permuted indices to shuffle all three arrays in the same way
side_img_tensor = side_img_tensor[permuted_indices]
top_img_tensor = top_img_tensor[permuted_indices]
labels = labels[permuted_indices]

# Save numpy binaries
np.save('../side_images.npy',side_img_tensor)
np.save('../top_images.npy',top_img_tensor)
np.save('../labes.npy',labels)

In [ ]:
# THIRD SCRIPT, TO READ IN THE NUMPY BINARIES, AND SET UP AND TRAIN THE MODEL

import pickle
import numpy as np
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, History
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Concatenate, Dense, Dropout
from tensorflow.keras.models import Model
from sys import argv, stderr

# Script accepts one argument, an integer in (0,1,2) representing which single label to use
which = int(argv[1])
outname = [ 'calc', 'mass', 'malignant' ][which]

print('Reading the tensors from disk...',file=stderr)

# Read all numpy binaries
side_img_tensor = np.load('side_images.npy')
top_img_tensor = np.load('top_images.npy')
labels = np.load('labes.npy')

# Getting just one of the 3 labels, since training 3 non-mutually exclusive labels gave issues
labels = labels[:, which].reshape(-1, 1)

# Calculate and print the mean value of the labels, which also represents the proportion of 1's vs 0's (0.50 = equal split)
mean_label = np.mean(labels)
print(f'Label mean: {mean_label}')

print('Setting up training and validation sets...',file=stderr)

# Calculate the index representing an 80:20 split of the data points
split_point = int(round(side_img_tensor.shape[0] * 0.80))

# Split the data; x_train and x_val are two element lists, the elements being side and top image tensors
x_train = [side_img_tensor[:split_point],top_img_tensor[:split_point]]
x_val = [side_img_tensor[split_point:],top_img_tensor[split_point:]]
y_train = labels[:split_point]
y_val = labels[split_point:]

print('Shapes:',file=stderr)
for x in [x_train,x_val]:
    for t in x:
        print(t.shape,file=stderr)

for y in [y_train,y_val]:
    print(y.shape,file=stderr)
    
print('\nDTypes:',file=stderr)
for x in [x_train,x_val]:
    for t in x:
        print(t.dtype,file=stderr)

for y in [y_train,y_val]:
    print(y.dtype,file=stderr)

print('\nSetting up the model...',file=stderr)

# NETWORK STARTS WITH 2 INPUTS, 2 PATHS

# Define input layers
side_view_input = Input(shape=(800, 800, 1), name='input_image_side')
top_view_input = Input(shape=(800, 800, 1), name='input_image_top')

# First convolutional layers for both images
side_conv_1 = Conv2D(32, kernel_size=(8,8), strides=2, padding='same', activation='relu')(side_view_input)
top_conv_1 = Conv2D(32, kernel_size=(8,8), strides=2, padding='same', activation='relu')(top_view_input)

# Pooling layer receives a tensor that is 400x400, and outputs 200x200
side_pool_1 = MaxPooling2D(pool_size=(2,2),padding='same')(side_conv_1)
top_pool_1 = MaxPooling2D(pool_size=(2,2),padding='same')(top_conv_1)

# EACH PATH BRANCHES HERE, FOR FOUR PATHS

# Second convolution layer A - 5x5 filters
side_conv_2A = Conv2D(32, kernel_size=(5,5), strides=2, padding='same', activation='relu')(side_pool_1)
top_conv_2A = Conv2D(32, kernel_size=(5,5), strides=2, padding='same', activation='relu')(top_pool_1)

# Second convolution layer B - 10x10 filters
side_conv_2B = Conv2D(32, kernel_size=(10,10), strides=2, padding='same', activation='relu')(side_pool_1)
top_conv_2B = Conv2D(32, kernel_size=(10,10), strides=2, padding='same', activation='relu')(top_pool_1)

# Global max pooling layer for both images (alternative to flattening, reduces size to number of filters)
side_global_pool_A = GlobalMaxPooling2D()(side_conv_2A)
side_global_pool_B = GlobalMaxPooling2D()(side_conv_2B)
top_global_pool_A = GlobalMaxPooling2D()(top_conv_2A)
top_global_pool_B = GlobalMaxPooling2D()(top_conv_2B)

# THE FOUR NETWORK PATHS CONVERGE TO ONE HERE

# Merge flattened image layers and other features
merged = Concatenate()([side_global_pool_A,side_global_pool_B,top_global_pool_A,top_global_pool_B])

# Output layer
output = Dense(1, activation='sigmoid')(merged)

# model = Model(inputs=[side_view_input,top_view_input,features_input],outputs=output)
model = Model(inputs=[side_view_input,top_view_input],outputs=output)

# Add L2 regularization to all layers capable of regularization (hasattr kernel_regularizer)
reg = regularizers.l1(0.01)
for layer in model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        layer.kernel_regularizer = reg     

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
best_model_checkpoint = ModelCheckpoint(f'best_model_{outname}.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=False, mode='max')
history = History()

print('Training the model...',file=stderr)

model.fit(x=x_train, y=y_train,
          validation_data=(x_val, y_val),
          epochs=50, batch_size=100, callbacks=[best_model_checkpoint, history])


# Save the training history to disk
with open(f'history_{outname}.pkl', 'wb') as f:
    pickle.dump(history.history, f)


